In [61]:
import os
import json
import matplotlib.pyplot as plt
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/ajay/Downloads/mykey.json"

### Extract sentiment of a text passed by the user

In [71]:
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

# Instantiates a client
client = language.LanguageServiceClient()

# The text to analyze
text = u'Tomorrow is going to be the end of the world'
document = types.Document(
    content=text,
    type=enums.Document.Type.PLAIN_TEXT)

# Detects the sentiment of the text
sentiment = client.analyze_sentiment(document=document).document_sentiment

print('Text: {}'.format(text))
print('Sentiment: {}, {}'.format(sentiment.score, sentiment.magnitude))

Text: Tomorrow is going to be the end of the world
Sentiment: -0.10000000149011612, 0.10000000149011612


In [30]:
from google.cloud import language_v1
from google.cloud.language_v1 import enums


def sample_analyze_sentiment(text_content):
    """
    Analyzing Sentiment in a String

    Args:
      text_content The text content to analyze
    """

    client = language_v1.LanguageServiceClient()

    # text_content = 'I am so happy and joyful.'

    # Available types: PLAIN_TEXT, HTML
    type_ = enums.Document.Type.PLAIN_TEXT

    # Optional. If not specified, the language is automatically detected.
    # For list of supported languages:
    # https://cloud.google.com/natural-language/docs/languages
    language = "en"
    document = {"content": text_content, "type": type_, "language": language}

    # Available values: NONE, UTF8, UTF16, UTF32
    encoding_type = enums.EncodingType.UTF8

    response = client.analyze_sentiment(document, encoding_type=encoding_type)
    # Get overall sentiment of the input document
    print(u"Document sentiment score: {}".format(response.document_sentiment.score))
    print(
        u"Document sentiment magnitude: {}".format(
            response.document_sentiment.magnitude
        )
    )
    # Get sentiment for all sentences in the document
    for sentence in response.sentences:
        print(u"Sentence text: {}".format(sentence.text.content))
        print(u"Sentence sentiment score: {}".format(sentence.sentiment.score))
        print(u"Sentence sentiment magnitude: {}".format(sentence.sentiment.magnitude))

    # Get the language of the text, which will be the same as
    # the language specified in the request or, if not specified,
    # the automatically-detected language.
    print(u"Language of the text: {}".format(response.language))


In [31]:
sample_analyze_sentiment("West Ham rocked Chelsea's top four hopes and boosted its survival bid with a dramatic 3-2 win, while Leicester lost 2-1 at Everton")

Document sentiment score: 0.800000011920929
Document sentiment magnitude: 0.800000011920929
Sentence text: West Ham rocked Chelsea's top four hopes and boosted its survival bid with a dramatic 3-2 win, while Leicester lost 2-1 at Everton
Sentence sentiment score: 0.800000011920929
Sentence sentiment magnitude: 0.800000011920929
Language of the text: en


### Extract sentiment of a text in a picture (need to use Vision API also here)

In [4]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    print('Texts:')

    for text in texts:
        print('\n"{}"'.format(text.description))

        vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in text.bounding_poly.vertices])

        print('bounds: {}'.format(','.join(vertices)))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [5]:
detect_text("C:/Users/ajay/Documents/leave letter.png")

Texts:

"AllBusiness
Templates
PERMISSION TO AVAIL LEAVE
Date: November 28, 2017
To,
ATTN
ADDRESS
POSTAL CODE
COUNTRY
Subject: permission to avail leave
Dear Mr./Mrs.,
I am writing this letter to ask your permission to avail annual leave for . Days (mention duration)
from . to .(mention dates) due to .(state reasons in brief). I will report back for work on
.(mention date of joining back).
I will hand over my duties and responsibilities to my colleague and team-mate, .(mention colleague's
name) and brief him about the works in hand.
I will be obliged if you approve my leave and get in touch in case I require any further extension of
leave. I will leave my contact details with your office in case of any queries.
Thanking You.
If you have any questions, please feel free to contact me directly. Looking forward to your reply.
Yours sincerely,
<NAME>
<Signature>
<Formal Name + Title>
Page 1 of 2
"
bounds: (94,69),(739,69),(739,1074),(94,1074)

"AllBusiness"
bounds: (690,69),(739,70),(739,77

In [13]:
from google.cloud import vision
import io
client = vision.ImageAnnotatorClient()

with io.open("C:/Users/ajay/Documents/nlp/img3.jpg", 'rb') as image_file:
    content = image_file.read()
image = vision.types.Image(content=content)
response = client.text_detection(image=image)

texts = response.text_annotations
texts1 = [text.description for text in response.text_annotations]
test=texts1[0]
test = test.replace('\n','')
print(test)

THE END


In [14]:
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

# Instantiates a client
client = language.LanguageServiceClient()

# The text to analyze
text = test
document = types.Document(
    content=text,
    type=enums.Document.Type.PLAIN_TEXT)

# Detects the sentiment of the text
sentiment = client.analyze_sentiment(document=document).document_sentiment

print('Text: {}'.format(text))
print('Sentiment: {}, {}'.format(sentiment.score, sentiment.magnitude))

Text: THE END
Sentiment: 0.30000001192092896, 0.30000001192092896


In [9]:
from google.cloud import vision
import io

def extract_text_fromimage(imagepath):
    client = vision.ImageAnnotatorClient()

    with io.open(imagepath, 'rb') as image_file:
        content = image_file.read()
        
    image = vision.types.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations
    texts1 = [text.description for text in response.text_annotations]
    test=texts1[0]
    test = test.replace('\n','')
    return test

### Sentiment of text from Google Storage

In [ ]:
from google.cloud import language_v1
from google.cloud.language_v1 import enums


def sample_analyze_sentiment(gcs_content_uri):
    """
    Analyzing Sentiment in text file stored in Cloud Storage

    Args:
      gcs_content_uri Google Cloud Storage URI where the file content is located.
      e.g. gs://[Your Bucket]/[Path to File]
    """

    client = language_v1.LanguageServiceClient()

    # gcs_content_uri = 'gs://cloud-samples-data/language/sentiment-positive.txt'

    # Available types: PLAIN_TEXT, HTML
    type_ = enums.Document.Type.PLAIN_TEXT

    # Optional. If not specified, the language is automatically detected.
    # For list of supported languages:
    # https://cloud.google.com/natural-language/docs/languages
    language = "en"
    document = {"gcs_content_uri": gcs_content_uri, "type": type_, "language": language}

    # Available values: NONE, UTF8, UTF16, UTF32
    encoding_type = enums.EncodingType.UTF8

    response = client.analyze_sentiment(document, encoding_type=encoding_type)
    # Get overall sentiment of the input document
    print(u"Document sentiment score: {}".format(response.document_sentiment.score))
    print(
        u"Document sentiment magnitude: {}".format(
            response.document_sentiment.magnitude
        )
    )
    # Get sentiment for all sentences in the document
    for sentence in response.sentences:
        print(u"Sentence text: {}".format(sentence.text.content))
        print(u"Sentence sentiment score: {}".format(sentence.sentiment.score))
        print(u"Sentence sentiment magnitude: {}".format(sentence.sentiment.magnitude))

    # Get the language of the text, which will be the same as
    # the language specified in the request or, if not specified,
    # the automatically-detected language.
    print(u"Language of the text: {}".format(response.language))


In [21]:
sample_analyze_sentiment('gs://buck910/gcp_nlp.txt')

Document sentiment score: 0.0
Document sentiment magnitude: 1.100000023841858
Sentence text: The move by The American Association for the Advancement of Science (AAAS), publisher of the esteemed Science journals, comes after Black scientists came forward to protest racism within academia and the sciences, and organized a strike on June 10th, that AAAS joined.
Sentence sentiment score: -0.5
Sentence sentiment magnitude: 0.5
Sentence text: In a letter to its 120,000 members this month, AAAS CEO Sudip Parikh announced that the 172-year-old institution has come up with a plan to hold itself accountable for making itself and the sciences more diverse.
Sentence sentiment score: 0.30000001192092896
Sentence sentiment magnitude: 0.30000001192092896
Sentence text: By September, it will start sharing data on the diversity of its staff, fellows, and authors published in its journals.
Sentence sentiment score: 0.20000000298023224
Sentence sentiment magnitude: 0.20000000298023224
Language of the te

### Sentiment of image from Google Storage

In [23]:
def extract_text_fromGSimage(GSimage_uri):
    client = vision.ImageAnnotatorClient()
    
    image = vision.types.Image()
    image.source.image_uri = GSimage_uri
    response = client.text_detection(image=image)
    texts = response.text_annotations
    texts1 = [text.description for text in response.text_annotations]
    test=texts1[0]
    test = test.replace('\n','')
    return test

In [25]:
extract_text_fromGSimage(GSimage_uri = "gs://buck910/ocr2.png")


'The quick brown foxjumped over the 5lazy dogs!'

In [26]:
# Instantiates a client
client = language.LanguageServiceClient()

# The text to analyze
text = test
document = types.Document(
    content=text,
    type=enums.Document.Type.PLAIN_TEXT)

# Detects the sentiment of the text
sentiment = client.analyze_sentiment(document=document).document_sentiment

print('Text: {}'.format(text))
print('Sentiment: {}, {}'.format(sentiment.score, sentiment.magnitude))

Text: THE END
Sentiment: 0.30000001192092896, 0.30000001192092896


###  Extracting Sentiment from a  PDF in google storage 

In [54]:
def Sent_PDF(gcs_source_uri, gcs_destination_uri):
    """OCR with PDF/TIFF as source files on GCS"""

    import re
    from google.cloud import vision
    from google.cloud import storage
    from google.protobuf import json_format
    # Supported mime_types are: 'application/pdf' and 'image/tiff'
    mime_type = 'application/pdf'

    # How many pages should be grouped into each json output file.
    batch_size = 1

    client = vision.ImageAnnotatorClient()

    feature = vision.types.Feature(
        type=vision.enums.Feature.Type.DOCUMENT_TEXT_DETECTION)

    gcs_source = vision.types.GcsSource(uri='gs://buck910/Review1.pdf')
    input_config = vision.types.InputConfig(
        gcs_source=gcs_source, mime_type=mime_type)

    gcs_destination = vision.types.GcsDestination(uri=gcs_destination_uri)
    output_config = vision.types.OutputConfig(
        gcs_destination=gcs_destination, batch_size=batch_size)

    async_request = vision.types.AsyncAnnotateFileRequest(
        features=[feature], input_config=input_config,
        output_config=output_config)

    operation = client.async_batch_annotate_files(
        requests=[async_request])

    print('Waiting for the operation to finish.')
    operation.result(timeout=420)

    # Once the request has completed and the output has been
    # written to GCS, we can list all the output files.
    storage_client = storage.Client()

    match = re.match(r'gs://([^/]+)/(.+)', gcs_destination_uri)
    bucket_name = match.group(1)
    prefix = match.group(2)

    bucket = storage_client.get_bucket(bucket_name)

    # List objects with the given prefix.
    blob_list = list(bucket.list_blobs(prefix=prefix))
    print('Output files:')
    for blob in blob_list:
        print(blob.name)

    # Process the first output file from GCS.
    # Since we specified batch_size=2, the first response contains
    # the first two pages of the input file.
    output = blob_list[0]

    json_string = output.download_as_string()
    response = json_format.Parse(
        json_string, vision.types.AnnotateFileResponse())

    # The actual response for the first page of the input file.
    first_page_response = response.responses[0]
    annotation = first_page_response.full_text_annotation

    # Here we print the full text from the first page.
    # The response contains more information:
    # annotation/pages/blocks/paragraphs/words/symbols
    # including confidence scores and bounding boxes
    print(u'Full text:\n{}'.format(
        annotation.text))
    sent_text = annotation.text
    sample_analyze_sentiment(sent_text)
    

In [55]:
Sent_PDF('gs://buck910/Review1.pdf','gs://buck910/Review1_out.json')

Waiting for the operation to finish.
Output files:
Review1_out.jsonoutput-1-to-1.json
Full text:
Cons
I would say the biggest issue that I have simply revolves around how much we've
grown in such a short amount of time. Being a new employee in Professional Services
has been tough and would have been impossible without my amazing team. There's so
MUCH information out there in various tools that there's just no way a person can
understand where to go and possibly consume it all. A lot of relevant information is
buried under irrelevant information. New tools come out, are exciting for a bit, then die
away. The second largest issue in my mind is that the goals and desires of the
Professional Services organization do not jive AT ALL with the goals and desires of my
particular client. Now, I know that I have tunnel vision and that I haven't had enough
experience with any other clients yet - I'm sure that with more time and exposure,
more insights will occur. But I really feel like the work t

###  Extracting Sentiment from a  PDF in local 

In [84]:
def print_result(annotations):
    score = annotations.document_sentiment.score
    magnitude = annotations.document_sentiment.magnitude

    for index, sentence in enumerate(annotations.sentences):
        sentence_sentiment = sentence.sentiment.score
        print('Sentence {} has a sentiment score of {}'.format(
            index, sentence_sentiment))
    
    print('Overall Sentiment: score of {} with magnitude of {}'.format(
        score, magnitude))
    return 0


def analyze(movie_review_filename):
    from google.cloud.vision import enums
    client = vision.ImageAnnotatorClient()
    
    mime_type = "application/pdf"
    with io.open(movie_review_filename, "rb") as f:
        content = f.read()
    input_config = {"mime_type": mime_type, "content": content}
    features = [{"type": enums.Feature.Type.DOCUMENT_TEXT_DETECTION}]

    pages = [1, 2, -1]
    requests = [{"input_config": input_config, "features": features, "pages": pages}]

    from google.cloud.language import enums
    response = client.batch_annotate_files(requests)
    for image_response in response.responses[0].responses:
        test=format(image_response.full_text_annotation.text)
    content = test.replace('\n','')
   
    client = language.LanguageServiceClient()
    document = types.Document(
        content=content,
        type=enums.Document.Type.PLAIN_TEXT)
    annotations = client.analyze_sentiment(document=document)
    
    # Print the results
    print(document)
    print_result(annotations)

In [85]:
analyze('C:\\Users\\ajay\\Documents\\nlp\\SampleTest.pdf')

type: PLAIN_TEXT
content: "What this book is aboutNeural networks are one of the most beautiful programming paradigms ever invented. In the conventional approach to programming, we tell the computer what to do, breaking big problems up into many small, precisely defined tasks that the computer can easily perform. By contrast, in a neural network we don\342\200\231t tell the computer how to solve our problem. Instead, it learns from observational data, figuring out its own solution to the problem at hand. Automatically learning from data sounds promising. However, until 2006 we didn\342\200\231t know how to train neural networks to surpass more traditional approaches, except for a few specialized problems. What changed in 2006 was the discovery of techniques for learning in so-called deep neural networks. These techniques are now known as deep learning. They\342\200\231ve been developed further, and today deep neural networks and deep learning achieve outstanding performance on many imp